# Sprint 4 - DEMO
## Team Network Training

# 1 Datan esikäsittely ja manipulointi

## 1.1 Sensoreista pois jääneet arvot nyt mukana

Viime viikolla havaittu blokkien jakolinjoille osuvien sensorimerkintöjen katoaminen, on nyt korjattu.

### 1.1.1 Viime viikon asetelma

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_e7e2ab822770a9b65385356926a33b25.png)

### 1.1.2 Tilanne nyt

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_e79e760634590f636e202d73be351d6a.png)

Ongelman kuvaus ja ratkaisu tarkemmin [CodiMD-dokumentaatiossa](https://gitlab.dclabra.fi/wiki/749Ptaf9QY6oxPqy1qpULQ?view#22-Blokkeihin-jaossa-ilmenneet-ongelmat).

## 1.2 Tiheyssensoreiden laskeminen tilanteissa, joissa osa sensorimerkinnöistä puuttuu

Viime viikolla havaittiin, että osa tiheyssensorimerkinnöistä puuttuu kokonaan. Tämä tuotti dataan rivejä, jotka sisälsivät NaN-arvoja, koska alkuperäinen laskentakaava ei ottanut huomioon puuttuvien sensoreiden tilannetta. Datan pelastamiseksi laadittiin `densityCheck()`-funktio, joka katsoo sensoreiden kokonaismäärän ja lisäksi uniikkien x-koordinaattien määrän. Tämän perusteella funktio arvioi, meneekö arkki läpi sellaisenaan, käynnistetäänkö pelastustoimet vai annetaanko arkille päätyä NaN-arvoja (jolloin arkki hylätään myöhemmin).

### 1.2.1 Esimerkkikuvia viime viikolla havaitusta ongelmasta

#### 1.2.1.1 Esimerkkitilanteita, jotka pelastetaan nykyään

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_a82dc09afebdd82adaaa51af77f0a225.png)
![image](https://gitlab.dclabra.fi/wiki/uploads/upload_fa1b71c22fe192c2a81950e5b370049f.png)
![image](https://gitlab.dclabra.fi/wiki/uploads/upload_b6c49d3ef7b13799192135595bd87d3c.png)

#### 1.2.1.2 Esimerkkitilanteita, joita ei oteta mukaan

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_0fc50aa97ef43aa3621b7866825b8929.png)
![image](https://gitlab.dclabra.fi/wiki/uploads/upload_448f8a5719213612a43eaca1732c468e.png)

### 1.2.2 Sanity check tiheyssensoreiden arvoille pelastusoperaation jälkeen

#### 1.2.2.1 Tiheysarvojen visualisointi

Pelastusoperaation jälkeen tehtiin niin sanottu sanity check -tiheyssensoreiden arvoille, jotta varmistuttaisiin, näyttävätkö ne järkeviltä. Alla havainnollistava kuva arvojen vaihtelusta.

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_ec07dd56debee084534150049a602dcf.png)

Pidempää pohdintaa kirjoitettiin tästäkin [CodiMD-dokumenttiin](https://gitlab.dclabra.fi/wiki/749Ptaf9QY6oxPqy1qpULQ?view#7-Tarkastellaan-density-arvoja).

### 1.2.2.2 Ongelmatapaus, joka tulee mukaan dataan

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_38d89bfbdf7e0a9e70f2f43c7ba7f189.png)

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_19908f73c3499bc2dcf4299792cac78c.png)

## 1.3 Väärään suuntaan "kutistuvat" arkit

Kun arkin leveyden ennustamisen lisäksi otettiin vaihdoehdoksi ennustaa kutistumaprosenttia, teimme samalla yllättävän havainnon datasta. Osa arkeista olikin alkuperäisen datan mukaan kasvanut kuivaamisen jälkeen. Nämä päätettiin pudottaa datasta pois ennen kuin data vietiin neuroverkolle.

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_ad95e29d3c093877e3ed2214df719df4.png)

Kun asetelmaa lähdettiin tutkimaan tarkemmin, huomattiin, että kaikki nämä seitsemän arkkia oli luokiteltu märkäarkkien datassa alimittaisiksi. Eli viilua on jatkettu, minkä takia sen lopullinen mitta on suurempi kuin märkänä kuvatessa.

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_a9aa2adc905b53dbed487facf2bc1430.png)

## 1.4 Datan augmentointi

Koska alkuperäistä dataa on vielä toistaiseksi aika vähän, lähdimme miettimään, voisiko datan määrää keinotekoisesti kasvattaa. Teimme kaksi asiaa:
- peilasimme blokkeja, jolloin vasen puoli vaihdettiin oikeaksi tai yläpuoli alapuoleksi tai sitten tehtiin molemmat yhtä aikaa
- lisäsimme dataan kohinaa

### 1.4.1 Blokkien peilaaminen

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_8cbc2d74a67ce0986945576df262f2a8.png)
![image](https://gitlab.dclabra.fi/wiki/uploads/upload_66c6a5082da7b167fab9d60b3bf2458e.png)
![image](https://gitlab.dclabra.fi/wiki/uploads/upload_521c5019dd3f087d0deece52a0f56215.png)
![image](https://gitlab.dclabra.fi/wiki/uploads/upload_7272db9499c9265f51b073c948497cce.png)

### 1.4.2 Kohinan lisääminen

```python
def addNoise(df_col):
    import numpy as np
    return df_col * np.random.choice([1.001,1.0015,0.999,0.9995],size=df_col.shape[0],p=[0.25,0.25,0.25,0.25])
```

Tällä hetkellä kohina on kovakoodattu promilletasolle, mutta voisi olla mielenkiintoista kokeilla antaa eri variaatioita parametrinä.

### 1.4.3 Augmentoidun datasetin koko

In [2]:
import pandas as pd
augmented = pd.read_pickle('/home/jovyan/work/raute_data/Augmented/augmentedNew.pkl')
augmented

,peelFile,m_uWidth,m_uLength,m_dThickness,B1MoistureAvg,B1TemperatureAvg,B1DensityAvg,B1KnotWidthSum,B1KnotCount,B1DecayWidthSum,...,B9DensityAvg,B9KnotWidthSum,B9KnotCount,B9DecayWidthSum,B9DecayCount,B9AllOtherDefectWidthSum,B9AllOtherDefectCount,dryFile,dryWidth,dryShrinkage
0,20210505123134_88.datx,1748.482417,1602.449015,1.56,96.235269,34.276972,884.334793,2.346144,0.9990,0.000000,...,892.958114,0.000000,0.0000,0.000000,0.000,0.000000,0.000,20210505173016_27.datx,1615.062604,0.924618
1,20210505150238_64.datx,1751.982883,1602.525485,1.56,82.772145,35.531013,813.685671,42.817124,7.9920,0.000000,...,814.212319,86.807321,15.9920,0.000000,0.000,0.000000,0.000,20210505160520_88.datx,1610.416550,0.921957
2,20210505141410_88.datx,1749.357534,1603.325947,1.56,95.082724,36.592860,885.221339,4.105752,0.9995,0.000000,...,888.311585,24.634510,1.9990,0.000000,0.000,0.000000,0.000,20210505162901_36.datx,1648.351455,0.941319
3,20210505150507_75.datx,1749.357534,1598.961639,1.56,87.700463,36.112723,838.967432,16.398397,2.9985,0.000000,...,870.738989,49.269020,6.9930,32.211136,1.999,0.000000,0.000,20210505160124_97.datx,1602.891365,0.914444
4,20210505135656_76.datx,1748.482417,1600.705327,1.56,79.387914,33.973388,791.311896,12.292645,2.0020,0.000000,...,800.300094,59.269721,8.0120,0.000000,0.000,0.000000,0.000,20210505163554_97.datx,1623.974048,0.927398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411543,20210505124057_64.datx,1749.357534,1596.999882,1.56,96.545771,33.669172,878.738472,158.048293,17.9820,13.497065,...,839.907332,160.204297,18.9905,0.000000,0.000,0.000000,0.000,20210505170258_17.datx,1501.838173,0.855937
411544,20210505123334_85.datx,1748.482417,1598.961639,1.56,113.846771,33.063095,997.013031,125.267907,10.9945,26.980654,...,1029.719285,157.462929,19.0190,39.297909,1.998,16.975557,1.998,20210505172657_52.datx,1635.539361,0.934003
411545,20210505150458_66.datx,1749.357534,1597.799181,1.56,89.055094,34.579096,855.408446,9.370512,2.0030,0.000000,...,895.952239,0.000000,0.0000,0.000000,0.000,0.000000,0.000,20210505160200_10.datx,1617.910546,0.922090
411546,20210505133012_23.datx,1751.982883,1600.779180,1.56,91.902511,35.601082,894.209977,0.000000,0.0000,12.884455,...,910.019476,0.000000,0.0000,0.000000,0.000,0.000000,0.000,20210505165613_81.datx,1642.725510,0.939044


In [3]:
augmented3 = pd.read_pickle('/home/jovyan/work/raute_data/Augmented/augmented3.pkl')
augmented3

,peelFile,m_uWidth,m_uLength,m_dThickness,B1MoistureAvg,B1TemperatureAvg,B1DensityAvg,B1KnotWidthSum,B1KnotCount,B1DecayWidthSum,...,B9DensityAvg,B9KnotWidthSum,B9KnotCount,B9DecayWidthSum,B9DecayCount,B9AllOtherDefectWidthSum,B9AllOtherDefectCount,dryFile,dryWidth,dryShrinkage
0,20210505124912_19.datx,1749.357534,1599.904574,1.56,82.447348,33.811381,864.473364,5.270913,0.999,0.000000,...,886.195101,5.278824,1.0010,6.451895,1.001,0.000000,0.0000,20210505164733_91.datx,1594.951084,0.909459
1,20210505143111_40.datx,1751.982883,1600.485512,1.56,111.142467,35.970355,978.324682,128.515535,8.991,122.647245,...,927.653012,89.153464,4.9950,0.000000,0.000,0.000000,0.0000,20210505162351_23.datx,1630.268703,0.933323
2,20210505124026_38.datx,1752.857999,1600.124098,1.56,97.993367,35.657107,873.168870,122.060416,10.015,0.000000,...,912.393366,72.766787,5.9970,68.624706,5.005,0.000000,0.0000,20210505170608_45.datx,1655.974420,0.943783
3,20210505130128_1.datx,1752.857999,1601.286556,1.56,103.745730,35.678622,898.282152,112.446149,8.008,0.000000,...,875.242352,36.896393,7.0105,0.000000,0.000,126.691765,0.9995,20210505171030_87.datx,1606.566229,0.920213
4,20210505150118_98.datx,1752.857999,1601.286556,1.56,92.842812,36.003405,891.462245,45.073032,6.006,0.000000,...,879.889350,70.829050,3.9980,0.000000,0.000,24.597595,2.0020,20210505161313_53.datx,1596.282603,0.913409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822543,20210505124646_99.datx,1751.982883,1600.124098,1.56,92.852889,35.199727,811.725860,78.048247,9.009,0.000000,...,850.417906,132.877639,13.0130,17.569711,1.001,28.754617,1.9980,20210505165340_23.datx,1643.945928,0.939273
822544,20210505123038_39.datx,1751.982883,1601.647389,1.56,107.481283,34.922863,887.468750,24.634510,4.004,0.000000,...,894.181036,4.105752,1.0015,0.000000,0.000,0.000000,0.0000,20210505173252_83.datx,1636.871555,0.935698
822545,20210505150803_8.datx,1748.482417,1598.380410,1.56,98.167038,33.591947,943.423893,10.557647,1.999,0.000000,...,898.687834,8.780461,0.9995,0.000000,0.000,0.000000,0.0000,20210505155517_69.datx,1611.300556,0.920161
822546,20210505135446_12.datx,1752.857999,1604.552082,1.56,84.714040,35.553526,794.531244,97.413602,10.010,0.000000,...,821.229889,70.829050,7.9920,0.000000,0.000,0.000000,0.0000,20210505164153_76.datx,1607.082903,0.920048


# 2 Tietokanta

- Tunkattu dockeriin ja käytetty Dockerfileä
- Features-taulu, target-taulu, verifiedpairs, VIEW, jossa näkyy kaikki
    - ei hyväksy duplikaatteja
- Mahdollisuus puskea myös ihan kaikki samaan tauluun duplikaatteineen (jos esim. augmentoitu)
    - Testattu 411k datasetillä, jos puskee kaiken kerralla menee ~3-4min (tässä siis piirrematriisi jo luotu)
- Työn alla datan puskeminen chunkeissa
    - Tämän hetkinen vaihtoehto on puskea joko rivi kerrallaan tai kaikki kerrallaan, silloin kun kyseessä on todellinen tilanne.
    - Augmentoidulle datasetille tehtiin testiloop, joka puskee augmentoitua dataa sisään chunkeissa.
        - 411k setillä ja 10k chunkeissa meni reilu 3 minuuttia.

Dokumentointi löytyy jälleen CodiMD:stä: [MariaDB-kontin pystyttäminen ja tietokantayhteyden muodostaminen](https://gitlab.dclabra.fi/wiki/Lhfjb-cJRFaSA_bvMb5eUg) ja [Kontin pystyttäminen Dockerfilen avulla](https://gitlab.dclabra.fi/wiki/QbEaq9AKSPmc3485Lk1mfw).

## 2.1 VIEW, jolla taulut saadaan yhdistettyä ja päivittymään automaattisesti

Piirrematriisi ja target-arvot on tässä erillisissä tauluissa. Linkkitauluna käytetään taulua, jossa on tiedot pareista.

```sql
CREATE VIEW PreprocessedData AS 
  (SELECT peelData.*, 
          dryData.* 
FROM peelData 
INNER JOIN VerifiedPairs ON peelData.peelFile = VerifiedPairs.pfile
INNER JOIN dryData ON VerifiedPairs.dfile = dryData.dryFile
          );
```

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_3eb5a5257251dda4a660fecb566c0f9e.png)

# 3 Neuroverkko

Viime viikolla saatiin ensimmäinen neuroverkon prototyyppi muodostettua, mutta ennusteet näyttivät vielä mitä sattuu. Tästä viimeviikkoisesta, tosin hieman muokatusta versiosta, tehtiin viikon alussa myös `SavedModel`, jotta Team 2 pääsee tekemään kokeiluja todellisella, tai sitä lähempänä olevalalla neuroverkolla.

Myöhemmin tällä viikolla ennusteet saatiin lähemmäs todellisuutta, vaikkakin alkuperäisellä datalla tehtynä ennusteissa oli vielä paljon heittoa. Augmentoidun datan ottaminen mukaan vei ennusteet jo hyvin lähelle target-arvoja. Hoksasimme myös, että kenties viime viikollakin tarkkuudet olisi saatu lähemmäs todellista skaalaa, ellemme olisi itse prediction vaiheessa tehneet vahingossa pientä virhettä.

Lisäksi tarkkuutta visualisoitiin opetuksen jälkeen vertaamalla niitä todellisiin arvoihin. Tarkkuudelle pyrittiin myös miettimään uusia mittareita, mutta vielä ei keksitty tyydyttävää ratkaisua.

## 3.1 Neuroverkko, jolla tarkkuudet saatiin lähemmäksi

```python
def reg_nn():
    
    model = models.Sequential()
    
    model.add(layers.Dense(128, activation='relu', input_shape=(transformed_big_train_data.shape[1],)))    
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1))
    
    # MSE Loss & MAE Loss - Regression task
    model.compile(optimizer='rmsprop', 
                  loss='mse', 
                  metrics=['mae']) 
    
    return model
```

## 3.2 Kustomoitujen metriikkojen lisääminen ja testaaminen

### 3.2.1 Funktiot metriikoille

```python
def distance_from_true(y_true, y_pred):
    return y_pred - y_true

def absolute_from_true(y_true, y_pred): # Täsmälleen sama kuin Mean Absolute Error
    return abs(y_pred - y_true)

def percentage_from_true(y_true, y_pred):
    return y_pred/y_true
```

### 3.2.1 Metriikkojen antaminen neuroverkolle

Metriikat annetaan neuroverkolle lisäämällä ne `model.compile`-osioon:

```python
network.compile(optimizer='rmsprop', loss='mse', metrics=['mae',distance_from_true,absolute_from_true,percentage_from_true])
```

## 3.3 Neuroverkon opetus alkuperäiskokoisella datasetillä

- Pudotimme `m_dThickness`-sarakkeen pois, koska sen arvo ei muutu kertaakaan koko datassa.
- Opetus- ja testijaoksi asetettiin 80/20.
- Data standardoitiin `sklearn`-kirjaston `StandarScalerin` avulla.

### 3.3.1 Opetus - original

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_6f58f892f7f4ef59ee8c068485485d15.png)

### 3.3.2 Validointi - original

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_846576dd4fc18f6e9069ccbe43afcddb.png)

### 3.3.3 Ennusteiden visualisointi - original

![](https://gitlab.dclabra.fi/wiki/uploads/upload_64de5266af5e444bbd1884e6b48e7e8b.png)

![](https://gitlab.dclabra.fi/wiki/uploads/upload_bdd8e803d49db3e7678cbec2e7427920.png)

![](https://gitlab.dclabra.fi/wiki/uploads/upload_16811dcb940145eb445a442c1f949013.png)

### 3.4 Neuroverkon opetus augmentoidulla datalla

Tällä kertaa pudotimme `m_dThickness`-sarakkeen pois, koska sen arvo ei muutu kertaakaan koko datassa. Opetus- ja testijaoksi asetettiin 70/30. Data standardoitiin `sklearn`-kirjaston `StandarScalerin` avulla.

### 3.4.1 Opetus - augmentoitu

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_361dfb7756be61f02f5b2d2d8348d127.png)

### 3.4.2 Validointi - augmentoitu

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_24028797b0d76551e04882f3838bad32.png)

### 3.4.3 Ennusteiden visualisointi - augmentoitu - ensimmäiset 100

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_0a0b4ff772d6954722b3a1942600c518.png)
![image](https://gitlab.dclabra.fi/wiki/uploads/upload_eaee1a2e378e03aa03cd9ca43f8fb677.png)
![image](https://gitlab.dclabra.fi/wiki/uploads/upload_49ba1a67b4be9313ce73e92417c05ba3.png)
![image](https://gitlab.dclabra.fi/wiki/uploads/upload_09470a865235cdb5bc68d7baa6c0476c.png)